In [1]:
"""
Se modela un problema de modelado de agentes reactivos, se representa a robots 
limpiadores (agentes reactivos) que viven dentro de una habitación de MxN 
espacios, que irá recorriendo la habitación limpiando basura. Se utiliza 
la librería Mesa.

Autores: Héctor González Sánchez (A01753863) y Alfredo Azamar López (A01798100)

Noviembre 10, 2023
"""

from mesa import Agent, Model 
from mesa.time import RandomActivation
from mesa.space import MultiGrid 
from mesa.datacollection import DataCollector 
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.experimental import JupyterViz

import solara
from matplotlib.figure import Figure

import random
import time

<IPython.core.display.Javascript object>

In [2]:
class RobotAgent(Agent):
    def __init__(self, uniqueId, model):
        """
        Constructor de la clase RobotAgent.

        Parámetros:
        - uniqueId: Identificador único del agente.
        - model: Modelo en el que el agente existe.

        Valor de retorno:
        Ninguno.
        """
        super().__init__(uniqueId, model)
        self.steps = 0

    def move(self):
        """
        Mueve al agente a una posición vecina de manera aleatoria.

        Parámetros:
        Ninguno.

        Valor de retorno:
        Ninguno.
        """
        possibleMoves = self.model.grid.get_neighborhood(
            self.pos, 
            moore=True, 
            include_center=False)
        newPosition = self.random.choice(possibleMoves)
        self.model.grid.move_agent(self, newPosition)

    def step(self):
        """
        Realiza un paso de simulación para el agente.

        Parámetros:
        Ninguno.

        Valor de retorno:
        Ninguno.
        """
        self.steps += 1
        cellContents = self.model.grid.get_cell_list_contents([self.pos])
        dirtyCell = next((agent for agent in cellContents if isinstance(agent, DirtyCellAgent)), None)

        if dirtyCell:
            self.model.grid.remove_agent(dirtyCell)
            self.model.dirtyCell -= 1
        else:
            self.move()

class DirtyCellAgent(Agent):
    def __init__(self, uniqueId, model):
        """
        Constructor de la clase DirtyCellAgent.

        Parámetros:
        - uniqueId: Identificador único del agente.
        - model: Modelo en el que el agente existe.

        Valor de retorno:
        Ninguno.
        """
        super().__init__(uniqueId, model)

    def step(self):
        """
        Realiza un paso de simulación para el agente.

        Parámetros:
        Ninguno.

        Valor de retorno:
        Ninguno.
        """
        pass

In [3]:
class CleanerModel(Model):
    def __init__(self, width, height, numAgents, dirtPercentage, maxSteps):
        """
        Constructor de la clase CleanerModel.

        Parámetros:
        - width: Ancho de la cuadrícula del modelo.
        - height: Altura de la cuadrícula del modelo.
        - numAgents: Número de agentes limpiadores en el modelo.
        - dirtPercentage: Porcentaje de celdas sucias en la cuadrícula inicial.
        - maxSteps: Número máximo de pasos de simulación.

        Valor de retorno:
        Ninguno.
        """
        self.numAgents = numAgents
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, False)
        self.maxSteps = maxSteps
        self.steps = 0
        self.running = True
        numDirtyCells = int((dirtPercentage / 100) * (width * height))
        self.dirtyCell = numDirtyCells

        for i in range(self.numAgents):
            agent = RobotAgent(i, self)
            self.schedule.add(agent)
            x = 1
            y = 1
            self.grid.place_agent(agent, (x, y))

        for i in range(numDirtyCells):
            DirtyCells = DirtyCellAgent(self.numAgents + i, self)
            self.schedule.add(DirtyCells)
            x, y = random.choice(list(self.grid.empties))
            self.grid.place_agent(DirtyCells, (x, y))

        self.datacollector = DataCollector(
            model_reporters={"Dirty Cells": lambda m: m.dirtyCell, "Cleaner Agents": lambda m: m.numAgents}
        )

    def step(self):
        """
        Realiza un paso de simulación para el modelo.

        Parámetros:
        Ninguno.

        Valor de retorno:
        Ninguno.
        """
        self.datacollector.collect(self)
        self.schedule.step()
        self.steps += 1
        if self.steps >= self.maxSteps or self.dirtyCell == 0:
            self.running = False

In [4]:
def make_agent_counts_chart(model):
    """
    Crea un gráfico de líneas que muestra la cantidad de agentes limpiadores y celdas sucias a lo largo de los pasos.

    Parámetros:
    - model: Instancia del modelo CleanerModel.

    Valor de retorno:
    - solara.FigureMatplotlib: Objeto que contiene el gráfico creado.
    """
    modelData = model.datacollector.get_model_vars_dataframe()
    fig = Figure()
    ax = fig.subplots()
    ax.plot(modelData.index, modelData['Cleaner Agents'], label='Cleaner Robots')
    ax.plot(modelData.index, modelData['Dirty Cells'], 'o-', label='Dirty Cells')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Count')
    ax.legend()

    return solara.FigureMatplotlib(fig)

def make_cleaning_efficiency_chart(model):
    """
    Crea un gráfico de líneas que muestra la eficiencia de limpieza a lo largo de los pasos.

    Parámetros:
    - model: Instancia del modelo CleanerModel.

    Valor de retorno:
    - solara.FigureMatplotlib: Objeto que contiene el gráfico creado.
    """
    modelData = model.datacollector.get_model_vars_dataframe()
    fig = Figure()
    ax = fig.subplots()
    totalCells = model.grid.width * model.grid.height
    modelData['Clean_efficiency'] = 1 - (modelData['Dirty Cells'] / totalCells)
    ax.plot(modelData.index, modelData['Clean_efficiency'] , 'o-', label='Cleaning Efficiency')
    ax.set_xlabel('Steps')
    ax.set_ylabel('Cleaning Efficiency')
    ax.legend()
    
    return solara.FigureMatplotlib(fig)

In [5]:
def my_agent_portrayal(agent):
    """
    Define la representación visual de los agentes en la interfaz gráfica.

    Parámetros:
    - agent: Instancia de un agente en el modelo.

    Valor de retorno:
    - dict: Diccionario que especifica la apariencia visual del agente.
    """
    if isinstance(agent, DirtyCellAgent):
        return {
            "color": "tab:red",
            "size": 50
        }
    elif isinstance(agent, RobotAgent):
        return {
            "color": "tab:blue",
            "size": 50 
        } 

model_params = {
    "width": 10,
    "height": 10,
    "numAgents": {
        "type": "SliderInt",
        "value": 50,
        "label": "Number of Cleaner Robots:",
        "min": 10,
        "max": 100,
        "step": 1
    },
    "dirtPercentage": {
        "type": "SliderInt",
        "value": 20,
        "label": "Percentage of dirty cells:",
        "min": 10,
        "max": 100,
        "step": 1
    },
    "maxSteps": {
        "type": "SliderInt",
        "value": 100,
        "label": "Maximum number of steps:",
        "min": 10,
        "max": 1000,
        "step": 1
    }
}

page = JupyterViz(
    CleanerModel,
    model_params,
    measures = [make_cleaning_efficiency_chart,
               make_agent_counts_chart],
    name="Cleaner Model",
    agent_portrayal = my_agent_portrayal
)

page

Cannot show ipywidgets in text